In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Default precise config for generation, no randomness
DECODE_CFG = {
    "max_new_tokens": 128,
    "min_new_tokens": 50,
    "num_beams": 5,
    "length_penalty": 2,
    "repetition_penalty": 1.1,
    "no_repeat_ngram_size": 3,
    "do_sample": False,
    "temperature": 0.7, # ignored if do_sample=False
    "top_p": 0.9, # ignored if do_sample=False
}

# ----- Generate function (supports overrides) -----
def generate_masked_span(model, tokenizer, text, **override):
    cfg = {**DECODE_CFG, **override}  # merge defaults + overrides
    inputs = tokenizer(text, return_tensors="pt", truncation=True).to(DEVICE)
    ids = model.generate(**inputs, **cfg)
    return tokenizer.decode(ids[0], skip_special_tokens=True)


# ----- Load model & dataset -----
model = AutoModelForSeq2SeqLM.from_pretrained("full_model_best").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("full_model_best")
full_dataset = load_from_disk("masked_dataset_2")
test_dataset = full_dataset.train_test_split(0.1, seed=42)["test"]

# ----- Test predictions -----
for i in range(5):
    x = test_dataset[i]["input_text"]
    print("\n--- Input ---\n", x)
    print("\n--- Predicted ---\n", generate_masked_span(model, tokenizer, x))
    print("\n--- Target ---\n", test_dataset[i]["target_text"])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Input ---
 <ADD> <START_OUTLINE> <BOE> Tim, feeling adventurous, expresses his desire to explore a cave. <EOE> <BOE> Despite his mother's warnings about the cave's dangers, Tim decides to enter it alone. <EOE> <BOE> Inside the cave, Tim encounters steam and feels scared and lost. <EOE> <BOE> Tim cries for his mom but realizes he is lost and cannot find his way out. <EOE> <END_OUTLINE> <START_STORY> One day, a little boy named Tim was feeling adventurous. He wanted to enter a big, dark cave near his house. He told his mom, "I want to go in the cave!" His mom said, "No, Tim!. It's not safe. Stay with me." But Tim did not listen. He went into the cave all alone. Inside, he saw steam coming from the ground. It was very hot and scary. Tim was afraid and wanted to go back home. But he could not find his way out. [MASK_START] He cried and cried for his mom, but she could not hear him. Tim was lost in the cave forever. [MASK_END] <END_STORY>

--- Predicted ---
 He cried and cried all the 

In [3]:
# Balanced config for generation with some randomness
BALANCED_CFG = dict(
    max_new_tokens=128,
    min_new_tokens=50,
    num_beams=1,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    length_penalty=1.0,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
)

# ----- Test predictions -----
for i in range(5):
    x = test_dataset[i]["input_text"]
    print("\n--- Predicted ---\n", generate_masked_span(model, tokenizer, x, **BALANCED_CFG))
    print("\n--- Target ---\n", test_dataset[i]["target_text"])


--- Predicted ---
 His mom was sad and cried. Tim wished he had not played in the cave all alone. He wished his adventurous spirit was not as good as knowing. So, he stayed in the dark cave, but he could not find his way out.

--- Target ---
 He cried and cried for his mom, but she could not hear him. Tim was lost in the cave forever.

--- Predicted ---
 Tom liked to play with his toys and watch the shows at night. He would run to his bed and run to the window and turn on his light. But his mom and dad said no. She said it was too late. The window was not nice.

--- Target ---
 Tom was a little boy who did not like to sleep in his bed. He wanted to play with his toys, watch his shows, and eat his snacks all night. He did not listen to his mom and dad when they told him to go to his bed and close his eyes.

--- Predicted ---
 He worked at the party every day. Everyone was happy and having fun. But Tom did not mind. He knew he was fearful. He just wanted to have fun, but he also wanted 

In [ ]:
# Creative config for generation with more randomness
CREATIVE_CFG = dict(
    max_new_tokens=150,
    min_new_tokens=50,
    num_beams=1,
    do_sample=True,
    temperature=1.1,
    top_p=0.95,
    length_penalty=1.0,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
)

# ----- Test predictions -----
for i in range(5):
    x = test_dataset[i]["input_text"]
    print("\n--- Predicted ---\n", generate_masked_span(model, tokenizer, x, **CREATIVE_CFG))
    print("\n--- Target ---\n", test_dataset[i]["target_text"])


--- Predicted ---
 He cried and cried. His mom came to help, but she could not find him. Tim felt very bad for what he had done. The end. Eventually, the cave was empty, and Tim could not go in. There was no more adventure in the cave.

--- Target ---
 He cried and cried for his mom, but she could not hear him. Tim was lost in the cave forever.

--- Predicted ---
 Tom loved his bed and slept all night. He liked to play with his toys, watch his shows, and watch his films on the big screen. He had a lot of fun and missed his toys. But his mom and dad said it was too late.

--- Target ---
 Tom was a little boy who did not like to sleep in his bed. He wanted to play with his toys, watch his shows, and eat his snacks all night. He did not listen to his mom and dad when they told him to go to his bed and close his eyes.

--- Predicted ---
 Tom knew that working at the party would be fun. Instead, there were big surprises to see people and toy cars. Some people came and loved to see them in 